In [24]:
import json
import requests

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
fpl_json = r.json()

In [25]:
import pandas as pd

teams = dict([
    [
        team['id'],
        dict(
            name=team['name'],
            short_name=team['short_name']
        )
    ]
    for team in fpl_json['teams']
])

positions = dict([
    [
        position['id'],
        dict(
            name=position['singular_name'],
            short_name=position['singular_name_short'],
            max_players=position['squad_select']
        )
    ]
    for position in fpl_json['element_types']
])

df_players = pd.DataFrame(fpl_json['elements'])
df_players['form'] = df_players['form'].astype(float)
df_players['selected_by_percent'] = df_players['selected_by_percent'].astype(float)

players = dict([
    [
        player['id'],
        dict(
            first_name=player['first_name'],
            last_name=player['second_name'],
            team_id=player['team'],
            position_id=player['element_type'],
            cost=player['now_cost'],
            total_points=player['total_points'],
            form=int(player['form'] * 10),
            selected_by_percent=int(player['selected_by_percent'] * 10)
        )
    ]
    for _, player in df_players.iterrows()
])

max_total_points = df_players['total_points'].max()
max_form = int(df_players['form'].max() * 10)
max_selected_by_percent = int(df_players['selected_by_percent'].max() * 10)

In [26]:
print('max. total points\t: {}\nmax. form\t\t: {}\nmax. selected (%)\t: {}'
      .format(max_total_points, max_form/10, max_selected_by_percent/10))

max. total points	: 130
max. form		: 8.0
max. selected (%)	: 62.5


In [35]:
### INPUT ###
available_money = 99

# weights required for the optimization
w_total_points = 0.5
w_form = 0.4
w_selected_by_percent = 0.1
#############

In [42]:
available_money_10 = available_money * 10  # available money multiplied by ten


# OR tools
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver("SCIP")

# defining decision variable x for players
x = {}
for player_id in players:
    x[player_id] = solver.IntVar(0, 1, "x[%i]" % player_id)

# constraint: total costs of players cannot be greater than available money
solver.Add(
    sum(
        x[player_id] * player_info["cost"] for player_id, player_info in players.items()
    )
    <= available_money_10
)

# constraint: each team cannot have more than 3 persons
for team_id in teams.keys():
    solver.Add(
        sum(
            x[player_id]
            for player_id, player_info in players.items()
            if player_info["team_id"] == team_id
        )
        <= 3
    )

# constraint: each position must be filled by the specified number of persons of this particular position
for pos_id, pos_info in positions.items():
    solver.Add(
        sum(
            x[player_id]
            for player_id, player_info in players.items()
            if player_info["position_id"] == pos_id
        )
        == pos_info["max_players"]
    )

solver.Maximize(
    solver.Sum(
        x[player_id]
        * (
            (player_info["total_points"] * w_total_points / max_total_points)
            + (player_info["form"] * w_form / max_form)
            + (
                player_info["selected_by_percent"]
                * w_selected_by_percent
                / max_selected_by_percent
            )
        )
        for player_id, player_info in players.items()
    )
)

status = solver.Solve()

selected_players = {}
if status == pywraplp.Solver.OPTIMAL:
    for player_id, player_info in players.items():
        if x[player_id].solution_value() == 1:
            selected_players[player_id] = player_info


In [44]:
for player_id, player_info in selected_players.items():
    print(positions[player_info['position_id']]['short_name'], teams[player_info['team_id']]['short_name'], player_info['first_name'], player_info['last_name'])

GKP AVL Emiliano Martínez
MID AVL Jack Grealish
MID AVL Anwar El Ghazi
FWD EVE Dominic Calvert-Lewin
FWD LEI Jamie Vardy
DEF LEE Stuart Dallas
FWD LEE Patrick Bamford
MID MUN Bruno Miguel Borges Fernandes
GKP SOU Alex McCarthy
DEF SOU Kyle Walker-Peters
DEF SOU Jan Bednarek
MID TOT Heung-Min Son
DEF WHU Angelo Ogbonna
DEF WHU Aaron Cresswell
MID WHU Tomas Soucek
